<a href="https://colab.research.google.com/github/MartaGacek1/MonteCarloProject1/blob/main/MonteCarloProjekt1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projekt I - Wprowadzenie do symulacji i metod Monte Carlo

## Marta Gacek

## 1. Generatory liczb pseudolosowych

### 1.1 LCG ("Linear Congruential Generator")



In [2]:
# x0 - warunek początkowy
# M - wartość modulo
# a, c - parametry z rekurencyjnego wzoru
# n - ilość liczb pseudolosowych, które otrzymamy

def LCG(x0, a, c, M, n):
    result = []
    x = x0 % M
    result.append(x)
    for i in range(n-1):
        x = (a * x + c) % M
        result.append(x)
    return [j/M for j in result]

test_lcg=LCG(30, 1, 5, 13, 10)
print(test_lcg)

[0.3076923076923077, 0.6923076923076923, 0.07692307692307693, 0.46153846153846156, 0.8461538461538461, 0.23076923076923078, 0.6153846153846154, 0.0, 0.38461538461538464, 0.7692307692307693]


### 1.2 GLCG ("Generalized Linear Congruential Generator")

In [3]:
# liczba współczynników musi być równa k, czyli liczbie wartości początkowych x_i dla i=0,...,k-1
# initials - lista k warunków początkowych
# coeffs - współczynniki a_i dla i=0,...,k-1
# M - wartość modulo
# n - ilość liczb pseudolosowych, które otrzymamy

def GLCG(initials, coeffs, M, n):
    k = len(initials)
    if len(coeffs) != k:
        raise ValueError("The number of coefficients doesn't match the number of initial values.")
    lst = initials[:] # lista "robocza"
    result = lst[:] # tutaj będą ostatecznie zwracane liczby
    result = [i/M for i in result] # normalizacja
    for i in range(n - k): # bo pierwsze k z n liczb już mamy na początku
        x = sum(coeffs[j] * lst[-(j + 1)] for j in range(k)) % M
        result.append(x/M) # normalizacja
        lst.append(x)
        lst.pop(0)  # potrzebujemy k ostatnich wartości do wyznaczania następnych x
    return result

test_glcg=GLCG([30, 40, 50], [3, 7, 68], 2**10, 10)
print(test_glcg)

[0.029296875, 0.0390625, 0.048828125, 0.412109375, 0.234375, 0.908203125, 0.388671875, 0.4609375, 0.861328125, 0.240234375]


### 1.3 RC4

In [4]:
# K - "klucz" - liczby z przedziału {0,...,m-1}, w naszym przypadku m=32
# n - ilość liczb pseudolosowych, które otrzymamy
# S - permutacja zbioru m-elementowego
# m - wartość modulo

import matplotlib.pyplot as plt
import random as rd
import numpy as np

def KSA(K, m=32):
   S = list(range(m))
   j = 0
   for i in range(m):
       j = (j + S[i] + K[i % len(K)]) % m
       S[i], S[j] = S[j], S[i]
   return S

def PRGA(S, n, m=32):
   i = 0
   j = 0
   result = []
   for k in range(n):
       i = (i + 1) % m
       j = (j + S[i]) % m
       S[i], S[j] = S[j], S[i]
       Y = S[(S[i] + S[j]) % m]
       result.append(Y/m) # normalizacja
   return result

def RC4(K, n, m=32):
  s = KSA(K, m)
  return PRGA(s, n, m)


k1 = [x for x in range(100)]
test_rc4 = RC4(k1, 1000)
print(test_rc4)

[0.5, 0.5625, 0.71875, 0.625, 0.78125, 0.75, 0.15625, 0.8125, 0.125, 0.4375, 0.96875, 0.3125, 0.9375, 0.40625, 0.0625, 0.78125, 0.9375, 0.9375, 0.46875, 0.90625, 0.53125, 0.65625, 0.25, 0.84375, 0.375, 0.875, 0.1875, 0.59375, 0.59375, 0.96875, 0.71875, 0.625, 0.96875, 0.71875, 0.625, 0.75, 0.625, 0.96875, 0.9375, 0.6875, 0.6875, 0.96875, 0.9375, 0.28125, 0.0625, 0.3125, 0.59375, 0.0, 0.125, 0.0, 0.125, 0.90625, 0.75, 0.96875, 0.3125, 0.0625, 0.1875, 0.1875, 0.21875, 0.03125, 0.15625, 0.28125, 0.4375, 0.75, 0.0625, 0.71875, 0.1875, 0.25, 0.46875, 0.40625, 0.40625, 0.84375, 0.25, 0.125, 0.03125, 0.3125, 0.4375, 0.15625, 0.15625, 0.46875, 0.15625, 0.3125, 0.40625, 0.78125, 0.1875, 0.03125, 0.28125, 0.15625, 0.28125, 0.3125, 0.28125, 0.0625, 0.65625, 0.0, 0.71875, 0.5, 0.71875, 0.625, 0.4375, 0.28125, 0.09375, 0.28125, 0.59375, 0.25, 0.21875, 0.71875, 0.34375, 0.9375, 0.9375, 0.40625, 0.125, 0.125, 0.0, 0.59375, 0.84375, 0.71875, 0.34375, 0.53125, 0.21875, 0.46875, 0.6875, 0.65625, 0.625, 

### 1.4 Xorshift

In [5]:
# x0 - niezerowy warunek początkowy
# a, b, c - parametry określające przesunięcia bitowe
# n - ilość liczb pseudolosowych, które otrzymamy

def xorshift(x0, n, a=21, b=35, c=4):
    if x0 == 0:
        raise ValueError("Initial value must not be zero.")
    x = x0
    result = []
    for i in range(n):
        x ^= (x >> a)  # przesunięcie w prawo i operacja xor
        x ^= (x << b)  # przesunięcie w lewo i operacja xor
        x ^= (x >> c)  # przesunięcie w prawo i operacja xor
        result.append(x & 0xFFFFFFFF)  # ograniczenie liczby x do zakresu 32-bitowego
    result = [i/(2**32 - 1) for i in result] # normalizacja (dzielimy przez największą możliwą wartość wygenerowaną przez xorshift)
    return result

test_xorshift = xorshift(20, n=10)
print(test_xorshift)

[4.8894435178696745e-09, 0.12507659665427093, 0.34108384450457147, 0.3374680917098811, 0.3372721654216927, 0.09438133940435511, 0.8854625418049895, 0.19366147816033602, 0.4966507629716421, 0.5117596659138239]


# 2. Testy statystyczne

## 2.1 Test $\chi^2$

In [14]:
from scipy.stats import chisquare

# LCG

st1, p1 = chisquare(test_lcg)
print("Statystyka chi-kwadrat dla LCG:", st1)
print("P-wartość dla LCG:", p1)

# GLCG

st2, p2 = chisquare(test_glcg)
print("Statystyka chi-kwadrat dla GLCG:", st2)
print("P-wartość dla GLCG:", p2)

# RC4

st3, p3 = chisquare(test_rc4)
print("Statystyka chi-kwadrat dla RC4:", st3)
print("P-wartość dla RC4:", p3)

# Xorshift

st4, p4 = chisquare(test_xorshift)
print("Statystyka chi-kwadrat dla Xorshift:", st4)
print("P-wartość dla Xorshift:", p4)


Statystyka chi-kwadrat dla LCG: 1.7287449392712548
P-wartość dla LCG: 0.9950676054414066
Statystyka chi-kwadrat dla GLCG: 2.4974783103099734
P-wartość dla GLCG: 0.9809512308039489
Statystyka chi-kwadrat dla RC4: 174.28069431825557
P-wartość dla RC4: 1.0
Statystyka chi-kwadrat dla Xorshift: 1.789220327898648
P-wartość dla Xorshift: 0.9943792247941656


## 2.2 Test Kołmogorowa-Smirnowa

In [16]:
from scipy.stats import ks_1samp, uniform

# LCG

st1, p1 = ks_1samp(test_lcg, uniform.cdf)
print("Statystyka Kołmogorowa-Smirnowa dla LCG:", st1)
print("P-wartość dla LCG:", p1)

# GLCG

st2, p2 = ks_1samp(test_glcg, uniform.cdf)
print("Statystyka Kołmogorowa-Smirnowa dla GLCG:", st2)
print("P-wartość dla GLCG:", p2)

# RC4

st3, p3 = ks_1samp(test_rc4, uniform.cdf)
print("Statystyka Kołmogorowa-Smirnowa dla RC4:", st3)
print("P-wartość dla RC4:", p3)

# Xorshift

st4, p4 = ks_1samp(test_xorshift, uniform.cdf)
print("Statystyka Kołmogorowa-Smirnowa dla Xorshift:", st4)
print("P-wartość dla Xorshift:", p4)

Statystyka Kołmogorowa-Smirnowa dla LCG: 0.15384615384615385
P-wartość dla LCG: 0.9440524832543472
Statystyka Kołmogorowa-Smirnowa dla GLCG: 0.33906250000000004
P-wartość dla GLCG: 0.15783781780320938
Statystyka Kołmogorowa-Smirnowa dla RC4: 0.04625000000000001
P-wartość dla RC4: 0.026865442869328016
Statystyka Kołmogorowa-Smirnowa dla Xorshift: 0.3882403340861761
P-wartość dla Xorshift: 0.07232384661077196


## 2.3 Test serii

In [28]:
import numpy as np
from scipy.stats import norm

def runs_test(data):
    median = np.median(data)
    binary = [1 if i >= median else 0 for i in data]  # zamiana danych na ciąg binarny (według mediany)
    x = np.sum(binary)  # Liczba 1 (liczenie elementów)
    y = len(binary) - x  # Liczba 0
    n = x + y
    if x == 0 or y == 0:
        raise ValueError("Not enough diversity of data. We cannot perform this test.")
    runs = 1  # liczba serii
    for i in range(1, len(binary)):
        if binary[i] != binary[i - 1]:
            runs += 1
    expected = (2 * x * y) / n + 1  # wartość oczekiwana liczby serii
    sigma = np.sqrt((2 * x * y * (2 * x * y - n)) / (n**2 * (n - 1)))  # odchylenie standardowe
    z = (runs - expected) / sigma
    p_value = 2 * (1 - norm.cdf(abs(z)))                   # czy trzeba zmieniać na uniform?
    return {
        "Statystyka Z": z,
        "P-wartość": p_value,
        "Liczba serii": runs,
        "Mediana": median
    }

In [29]:
# LCG

lcg_data = runs_test(test_lcg)
print("Statystyka testu serii dla LCG:", lcg_data["Statystyka Z"])
print("P-wartość dla LCG:", lcg_data["P-wartość"])

# GLCG

glcg_data = runs_test(test_glcg)
print("Statystyka testu serii dla GLCG:", glcg_data["Statystyka Z"])
print("P-wartość dla GLCG:", glcg_data["P-wartość"])

# RC4

rc4_data = runs_test(test_rc4)
print("Statystyka testu serii dla RC4:", rc4_data["Statystyka Z"])
print("P-wartość dla RC4:", rc4_data["P-wartość"])

# Xorshift

xorshift_data = runs_test(test_xorshift)
print("Statystyka testu serii dla Xorshift:", xorshift_data["Statystyka Z"])
print("P-wartość dla Xorshift:", xorshift_data["P-wartość"])

Statystyka testu serii dla LCG: 1.3416407864998738
P-wartość dla LCG: 0.17971249487899987
Statystyka testu serii dla GLCG: -0.6708203932499369
P-wartość dla GLCG: 0.502334954360502
Statystyka testu serii dla RC4: -0.8839129955737252
P-wartość dla RC4: 0.376743181379843
Statystyka testu serii dla Xorshift: 0.0
P-wartość dla Xorshift: 1.0
